In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import os
import datetime as dt
import natsort 
import glob
plt.style.use('../../style.mplstyle')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
import matplotlib.colors as coloUrs
import matplotlib.cm as cmx
from scipy.signal import find_peaks

In [15]:
def get_color_map(n,name='viridis'):
    colormap=plt.get_cmap(name)
    cNorm= coloUrs.Normalize(vmin=0,vmax=n-1)
    scalarMap=cmx.ScalarMappable(norm=cNorm,cmap=colormap)
    outmap=[]
    for i in range(n):
        outmap.append(scalarMap.to_rgba(i))
    return outmap,scalarMap

In [16]:

def rebin(hx,h,bins):
    h_rebin=[]
    # hx_rebin=[]
    for i in range(int(len(h)/bins)):
        start_idx=i*bins
        end_idx=(i+1)*bins
        h_rebin.append(np.sum(h[start_idx:end_idx]))
        # hx_rebin.append(np.mean(hx[start_idx:end_idx]))
    hx_rebin=range(len(h_rebin))
    return np.array(hx_rebin), np.array(h_rebin)

In [17]:
def gauss(x,a,b,c):
    return a*np.exp(-(x-b)**2/(2*c**2))

def line(x,a,b):
    return a*(x-b)

In [20]:
gain_single_pe_0=[]
gain_err_single_pe_0=[]
intercept_0=[]
sumdata = 0
mu_cdf_0=[]

calib_pe_0=[]
calib_count_0=[]

mu_gauss_list_0V=[]
mu_gauss_err_0V=[]

BINS=10 #bins for rebinning
N_peaks=4
Path = "C:\\Users\\nEXO\\Downloads\\SiPM_response_LXe_0V\\"

Files = glob.glob(Path+"*.csv")
print('Number of files: {}'.format(len(Files)))


for j,f in enumerate(natsort.natsorted(Files)): 



    data = pd.read_csv(f, delimiter=',', skiprows=1)
    hx,h = rebin(np.array(data['Channel']), np.array(data['Counts']), BINS)
    
    h=h/60 #Divide by 60s to calculate the rate

    # f, ax = plt.subplots(3, 1, gridspec_kw={'height_ratios': [5, 3, 5]})
    # ax[0].set_xlim(0,200)
    # ax[0].set_yscale('log')
    # ax[0].step(hx,h, label='$E_\mathrm{{drift}} = {}\,$V/cm'.format(0), where='mid')
    # ax[0].set_title(str(j+1)+' Minutes')
    # p0 = hx_nof[np.where(h_nof == np.max(h_nof))][0]
    params = []

    PROMINENCE=10000 #This prominence is re-set here to ensure that every file starts out with a high prominence

    peaks,pdict=find_peaks(h,prominence=PROMINENCE,distance=5)

    peak_length=len(peaks)
   

    # print('peak_length before while loop: ',peak_length)

    #We want to ensure that using a high prominence gives us at least 3 peaks to fit a straight line to. If it doesn't we reduce prominence till we get at least 3 peaks. N_peaks is set above
    while (peak_length<N_peaks+1):
        PROMINENCE=PROMINENCE-1
        peaks,pdict=find_peaks(h,prominence=PROMINENCE,distance=5)
        peak_length=len(peaks)

    print('peak_length: ',peak_length)

    if peak_length!=5:
        print('PEAK_LENGTH !=5************************')

    # ax[0].plot(hx[peaks],h[peaks],'*') # plot the peak markers

    # first_pe_max=hx[peaks[1]] # The x-value of the 2nd peak.Index=1 means the second peak will be used for getting fit parameters
    # max_value=h[peaks[1]] # The height of the 2nd peak
    # x_idx_array=(h<0.5*max_value) & (hx>first_pe_max)# returns a boolean array where both conditions are true

    # right_side_x= hx[np.where(x_idx_array)[0][0]] #finding the first time where x_idx_array is True
        
    # sigma_guess=np.abs(first_pe_max-right_side_x) #We need this to fit the width of the Gaussian peak

    # cut= (hx < first_pe_max+sigma_guess) & (hx > first_pe_max-sigma_guess) # This cut helps to fix the width of the peak-fit

    # popt,pcov=curve_fit(gauss,hx[cut],h[cut],p0=[max_value,first_pe_max,sigma_guess]) # We use curve_fit to return the optimal parameters and the covariance matrix

    # ax[0].plot(hx[cut],gauss(hx[cut],*popt),color='green',label='Fit',lw=2,alpha=0.5) # Here we plot the fit on the 2nd peak


    # for i,peak in enumerate(peaks[1:]): #here we ignore the first peak because it could be the pedestal
        


    #     new_first_pe_max=hx[peak] #x-value of the peak


    #     new_max_value=h[peak] #height of the peak


        

    #     new_x_idx_array=(h<0.5*new_max_value) & (hx>new_first_pe_max) # returns a boolean array where both conditions are true


    #     new_right_side_x= hx[np.where(new_x_idx_array)[0][0]] #finding the first time where x_idx_array is True



    #     new_sigma_guess=np.abs(new_first_pe_max-new_right_side_x) #We need this to fit the width of the Gaussian peaks


    #     new_cut= (hx < new_first_pe_max+new_sigma_guess) & (hx > new_first_pe_max-new_sigma_guess) # This cut helps to fix the width of the peak-fit

    #     popt_new,pcov_new=curve_fit(gauss,hx[new_cut],h[new_cut],p0=[new_max_value,new_first_pe_max,new_sigma_guess],maxfev=100000) # We use curve_fit to return the optimal parameters and the covariance matrix

    #     ax[0].plot(hx[new_cut],gauss(hx[new_cut],*popt_new),color='r',label='Fit',lw=3) # Here we plot the fit on all the peaks

    #     params.append(popt_new)

       
    # gain = 12.0
    # for i in range(3):

    #     if(i>0): p0 = p0_max+gain

    #     # select events around that peak
    #     cut = np.where((hx_nof>p0-3) & (hx_nof<p0+3))

    #      #find position of max in the selected region
    #     p0_max = hx_nof[np.where(h_nof == np.max(h_nof[cut]))][0]

    #     # redefine selection cut based on true peak position
    #     cut = np.where((hx_nof>p0-4) & (hx_nof<p0+4))
    #     print(hx_nof[cut])

    #     # run and plot fit
    #     popt,pcov = curve_fit(gauss, hx_nof[cut], h_nof[cut], p0=[np.max(h_nof[cut]),p0,1],maxfev=100000)
    #     ax[0].plot(hx_nof[cut], gauss(hx_nof[cut], *popt), color=colors[1],lw=3)
    #     # ax[0].set_ylim(1E-5,1E-1)
    #     print(popt[1])
            
    #     # append best fit parameters
    #     params.append(popt)
    #     # print(popt)
    #     if(i==1):
    #         gain = np.array(params)[1,1] - np.array(params)[0,1]

    # ax[1].set_xlim(0,len(params)-1)
    # xdata = [i for i,x in enumerate(params)]
    # ax[1].scatter(xdata, np.array(params)[:,1], s=10, zorder=10)
    # popt_nof,pcov = curve_fit(line, xdata, np.array(params)[:,1], p0=[10,0] )
    # gain_single_pe_0.append(popt_nof[0])
    # intercept_0.append(popt_nof[1])
    # ax[1].plot(xdata, line(xdata, *popt_nof), color=colors[1], label='Gain: {:.2f} ADC/p.e.'.format(popt_nof[0]))
    
    # calib_pe=hx/popt_nof[0]+popt_nof[1]
    # ax[2].step(calib_pe,h, label='$E_\mathrm{{drift}} = {}\,$V/cm'.format(0))








    # ax[1].legend(loc='upper left')
    # ax[2].set_xlim(10,120)
    # ax[2].set_xlabel('P.E')
    # # ax[2].set_ylim(0,1E-3)
    # ax[2].set_yscale('log')
    # ax[2].legend(loc='upper right')

    # #append the calibrated data into a list of array

    # calib_pe_0.append(np.array(calib_pe))
    # calib_count_0.append(np.array(h_nof))

    # # Now find the cdf of the source bump
    # # cut_median=np.where((calib_pe>25) & (calib_pe<110))
    # cut_median=np.where((calib_pe>35) & (calib_pe<90))
    # cdf=np.cumsum(h_nof[cut_median])/np.sum(h_nof[cut_median])
    # mu_interp=np.interp(0.5,cdf,calib_pe[cut_median])

    # mu_idx=np.argmin(np.abs(calib_pe-mu_interp))
    # ax[2].plot(calib_pe[mu_idx],h_nof[mu_idx],'*')
    # mu_cdf_0.append(calib_pe[mu_idx])


    # #now we fit the gaussian to the source bump
    # cut_gauss=np.where((calib_pe>40) & (calib_pe<100))
    # popt_bump_0,pcov_bump_0=curve_fit(gauss,calib_pe[cut_gauss],h_nof[cut_gauss],p0=[np.max(h_nof[cut_gauss]),60,100],maxfev=100000)
    # ax[2].plot(calib_pe[cut_gauss],gauss(calib_pe[cut_gauss],*popt_bump_0),color='r',lw=5,label=str(popt_bump_0))
    # mu_gauss_list_0V.append(popt_bump_0[1])
    # mu_gauss_err_0V.append(np.sqrt(np.diag(pcov_bump_0))[1])





    # plt.tight_layout()
    # plt.show()

Number of files: 60
peak_length:  5
peak_length:  5
peak_length:  5
peak_length:  5
peak_length:  5
peak_length:  5
peak_length:  5
peak_length:  5
peak_length:  5
peak_length:  5
peak_length:  5
peak_length:  5
peak_length:  5
peak_length:  5
peak_length:  5
peak_length:  5
peak_length:  5
peak_length:  6
PEAK_LENGTH !=5************************
peak_length:  5
peak_length:  5
peak_length:  5
peak_length:  5
peak_length:  5
peak_length:  5
peak_length:  5
peak_length:  5
peak_length:  5
peak_length:  5
peak_length:  5
peak_length:  5
peak_length:  5
peak_length:  5
peak_length:  5
peak_length:  5
peak_length:  5
peak_length:  6
PEAK_LENGTH !=5************************
peak_length:  5
peak_length:  5
peak_length:  5
peak_length:  5
peak_length:  5
peak_length:  5
peak_length:  6
PEAK_LENGTH !=5************************
peak_length:  5
peak_length:  5
peak_length:  6
PEAK_LENGTH !=5************************
peak_length:  5
peak_length:  5
peak_length:  5
peak_length:  5
peak_length:  5
peak

In [ ]:
time_0V=np.arange(0,60,1)
plt.figure(figsize=(6,4))
plt.errorbar(time_0V,gain_single_pe_0,fmt='o')
plt.xlabel('Time [minutes]')
plt.ylabel('Gain [ADC/PE]')
plt.ylim(12.5,30)
plt.grid()
plt.savefig('gain_fluctuation.pdf')
plt.show()

In [ ]:

def rebin_(hx,h,bins):
    h_rebin=[]
    hx_rebin=[]
    for i in range(int(len(h)/bins)):
        start_idx=i*bins
        end_idx=(i+1)*bins
        h_rebin.append(np.sum(h[start_idx:end_idx]))
        hx_rebin.append(np.mean(hx[start_idx:end_idx]))
    # hx_rebin=range(len(h_rebin))
    return np.array(hx_rebin), np.array(h_rebin)

In [ ]:
plt.subplot(2,2,1)
plt.plot(calib_pe_0[0],calib_count_0[0],lw=3,label='1 Minute')
# plt.plot(calib_pe_0[59],calib_count_0[59],lw=3,label='60 Minutes')
cut=np.where((calib_pe_0[0]>0.75) & (calib_pe_0[0]<1.25))
popt,pcov=curve_fit(gauss,calib_pe_0[0][cut],calib_count_0[0][cut],p0=[np.max(calib_count_0[0][cut]),1,0.5])
plt.plot(calib_pe_0[0][cut],gauss(calib_pe_0[0][cut],*popt),color='r',lw=2,label='Fit')
# print(popt)
params_pe = [] 

for i in range(1,5):
    cut_pe=np.where((calib_pe_0[0]>i*popt[1]-0.25) & (calib_pe_0[0]<i*popt[1]+0.25))
    print(calib_pe_0[0][cut_pe])
    popt_pe,pcov_pe=curve_fit(gauss,calib_pe_0[0][cut_pe],calib_count_0[0][cut_pe],p0=[np.max(calib_count_0[0][cut_pe]),i*popt[1],popt[2]])
    plt.plot(calib_pe_0[0][cut_pe],gauss(calib_pe_0[0][cut_pe],*popt_pe),color='r',lw=2)
    params_pe.append(popt_pe)
    plt.xlim(0,10)
plt.subplot(2,2,2)
# xdata = [i for i,x in enumerate(params_pe)]
xdata=np.arange(1,len(params_pe)+1,1)
plt.scatter(xdata, np.array(params_pe)[:,1], s=10, zorder=10)
popt_line,pcov_line = curve_fit(line, xdata, np.array(params_pe)[:,1], p0=[10,0] )
x_plot=np.arange(0,xdata[-1]+1,1)
plt.plot(x_plot, line(x_plot, *popt_line), color=colors[1], label='Gain: {:.2f} ADC/p.e.'.format(popt_line[0]))

plt.legend(loc=2)
# plt.xlim(0,10)
plt.grid()
plt.xlabel('PE')
plt.ylabel('Counts')
plt.savefig('calibration.pdf')
plt.show()
# print(calib_pe_0[0][2]-calib_pe_0[0][1])

In [ ]:
pe_corrected_gain=[]
pe_corrected_gain_err=[]
for pe,count in zip(calib_pe_0,calib_count_0):
    plt.subplot(2,2,1)
    plt.plot(pe,count)
    plt.xlim(0,10)
    cut=np.where((pe>0.75) & (pe<1.25))
    popt,pcov=curve_fit(gauss,pe[cut],count[cut],p0=[np.max(count[cut]),1,0.5])
    plt.plot(pe[cut],gauss(pe[cut],*popt),color='r',lw=2,label='Fit')
    # print(popt)
    params_pe = [] 

    for i in range(1,5):
        cut_pe=np.where((pe>i*popt[1]-0.25) & (pe<i*popt[1]+0.25))
        # print(pe[cut_pe])
        popt_pe,pcov_pe=curve_fit(gauss,pe[cut_pe],count[cut_pe],p0=[np.max(count[cut_pe]),i*popt[1],popt[2]])
        plt.plot(pe[cut_pe],gauss(pe[cut_pe],*popt_pe),color='r',lw=2)
        params_pe.append(popt_pe)
        plt.xlim(0,10)
    plt.subplot(2,2,2)
    xdata=np.arange(1,len(params_pe)+1,1)
    plt.scatter(xdata, np.array(params_pe)[:,1], s=10, zorder=10)
    popt_line,pcov_line = curve_fit(line, xdata, np.array(params_pe)[:,1], p0=[10,0] )
    x_plot=np.arange(0,xdata[-1]+1,1)
    plt.plot(x_plot, line(x_plot, *popt_line), color=colors[1], label='Gain: {:.2f} ADC/p.e.'.format(popt_line[0]))
    pe_corrected_gain.append(popt_line[0])
    pe_corrected_gain_err.append(np.sqrt(np.diag(pcov_line)[0]))
    plt.legend(loc=2)
    # plt.xlim(0,10)
    plt.grid()
    plt.xlabel('PE')
    plt.ylabel('Counts')
    # plt.savefig('calibration.pdf')
    plt.show()
    

In [ ]:
plt.figure(figsize=(6,4))
plt.errorbar(time_0V,pe_corrected_gain,yerr=pe_corrected_gain_err,fmt='o')
plt.ylabel('Normalized PE/PE')
plt.xlabel('Time [minutes]')
plt.ylim(0.8,1.2)
plt.grid()
plt.savefig('corrected_gain.pdf')
plt.show()

In [ ]:


color_array,sm=get_color_map(len(calib_pe_0))
n_bins=1 # number of bins to rebin by
n_files=5 #number of files to add together
avg=0

plt.figure(figsize=(8,6))




for i in range(len(calib_pe_0)):

    x,y=rebin_(calib_pe_0[i],calib_count_0[i],n_bins)
    avg+=y
    if (i%n_files==n_files-1):

        # plt.subplot(2,1,1)
        # plt.plot(x,avg,lw=3,color=color_array[i])
        # plt.title('E= 0 V/cm')
        # # plt.yscale('log')
        # plt.xlim(20,120)
        # plt.ylim(0,90)
        # plt.ylabel('Counts')


        # plt.subplot(2,1,2)
        plt.plot(x,avg,lw=3,color=color_array[i])
        # plt.yscale('log')
        plt.xlim(0,10)
        # plt.ylim(0,100)
        plt.ylabel('Counts')
        avg=0
plt.colorbar(sm)
plt.xlabel('PE')
plt.grid()
plt.savefig('peak_shift0.pdf')
plt.show()

In [ ]:

plt.figure(figsize=(6,4))
plt.errorbar(time_0V,np.array(mu_gauss_list_0V),yerr=np.array(mu_gauss_err_0V),fmt='o',label='Gauss fit')
# plt.errorbar(time_0V,np.array(mu_exp_gauss_list_0V),yerr=np.array(mu_exp_gauss_err_0V),fmt='o',label='gauss+exp')

plt.grid()
plt.xlabel('Time[minutes]')
plt.ylabel('$\mu$ [PE]')
# plt.scatter(time_0V,mu_cdf_0,label='CDF',color=colors[2])
# plt.scatter(time_0V,np.array(gain_single_pe_0),label='Gain (arb units)',color=colors[3])
plt.legend(loc=2)
plt.savefig('mu_0.pdf')
plt.show()

In [ ]:
gain_single_pe_273=[]
intercept_273=[]
sumdata = 0

mu_cdf_273=[]

calib_pe_273=[]
calib_count_273=[]

mu_gauss_list_273V=[]
mu_gauss_err_273V=[]

BINS=10 #bins for rebinning


Path = "C:\\Users\\nEXO\\Downloads\\SiPM_response_LXe_273V\\"

Files = glob.glob(Path+"*.csv")
print('Number of files: {}'.format(len(Files)))

for j,f in enumerate(natsort.natsorted(Files)): 
    if j==128:
        continue
    else:


        data = pd.read_csv(f, delimiter=',', skiprows=1)
        hx_wf,h_wf = rebin(np.array(data['Channel']), np.array(data['Counts']), BINS)
        # h_wf = h_wf/np.sum(h_wf)
        h_wf=h_wf/60 #Divide by 60s to calculate the rate

        f, ax = plt.subplots(3, 1, gridspec_kw={'height_ratios': [5, 3, 5]})
        ax[0].set_xlim(0,200)
        ax[0].set_yscale('log')
        ax[0].step(hx_wf,h_wf, label='$E_\mathrm{{drift}} = {}\,$V/cm'.format(273), where='mid')
        ax[0].set_title(str(j+1)+' Minutes')
        p0 = hx_wf[np.where(h_wf == np.max(h_wf))][0]
        params = []
       
        gain = 12.0
        for i in range(3):

            if(i>0): p0 = p0_max+gain

            # select events around that peak
            cut = np.where((hx_wf>p0-3) & (hx_wf<p0+3))

            #find position of max in the selected region
            p0_max = hx_wf[np.where(h_wf == np.max(h_wf[cut]))][0]

            # redefine selection cut based on true peak position
            cut = np.where((hx_wf>p0-4) & (hx_wf<p0+4))
            print(hx_wf[cut])

            # run and plot fit
            popt,pcov = curve_fit(gauss, hx_wf[cut], h_wf[cut], p0=[np.max(h_wf[cut]),p0,1],maxfev=100000)
            ax[0].plot(hx_wf[cut], gauss(hx_wf[cut], *popt), color=colors[1],lw=3)
            # ax[0].set_ylim(1E-5,1E-1)
            print(popt[1])
            
            # append best fit parameters
            params.append(popt)
            # print(popt)
            if(i==1):
                gain = np.array(params)[1,1] - np.array(params)[0,1]
        ax[0].legend(loc='upper right')
        # ax[1].set_xlim(0,9)
        xdata = [i for i,x in enumerate(params)]
        ax[1].scatter(xdata, np.array(params)[:,1], s=10, zorder=10)
        popt_wf,pcov = curve_fit(line, xdata, np.array(params)[:,1], p0=[10,0] )
        gain_single_pe_273.append(popt_wf[0])
        intercept_273.append(popt_wf[1])
        ax[1].plot(xdata, line(xdata, *popt_wf), color=colors[1], label='Gain: {:.2f} ADC/p.e.'.format(popt_wf[0]))
        ax[1].legend(loc='upper left')
        calib_pe=hx_wf/popt_wf[0]+popt_wf[1]
        ax[2].step(calib_pe,h_wf, label='$E_\mathrm{{drift}} = {}\,$V/cm'.format(273))
        ax[2].set_xlim(10,120)
        ax[2].set_xlabel('P.E')
        # ax[2].set_ylim(0,1E-3)
        ax[2].set_yscale('log')
        ax[2].legend(loc='upper right')

        #append the calibrated data into a list of array

        calib_pe_273.append(np.array(calib_pe))
        calib_count_273.append(np.array(h_wf))

        # Now find the cdf of the source bump
        # cut_median=np.where((calib_pe>25) & (calib_pe<110))
        cut_median=np.where((calib_pe>35) & (calib_pe<90))
        cdf=np.cumsum(h_wf[cut_median])/np.sum(h_wf[cut_median])
        mu_interp=np.interp(0.5,cdf,calib_pe[cut_median])

        mu_idx=np.argmin(np.abs(calib_pe-mu_interp))
        ax[2].plot(calib_pe[mu_idx],h_wf[mu_idx],'*')
        mu_cdf_273.append(calib_pe[mu_idx])


        #now we fit the gaussian to the source bump
        cut_gauss=np.where((calib_pe>20) & (calib_pe<90))
        popt_bump_273,pcov_bump_273=curve_fit(gauss,calib_pe[cut_gauss],h_wf[cut_gauss],p0=[np.max(h_wf[cut_gauss]),60,100],maxfev=100000)
        ax[2].plot(calib_pe[cut_gauss],gauss(calib_pe[cut_gauss],*popt_bump_273),color='r',lw=5,label=str(popt_bump_273))
        mu_gauss_list_273V.append(popt_bump_273[1])
        mu_gauss_err_273V.append(np.sqrt(np.diag(pcov_bump_273))[1])


        plt.tight_layout()
        plt.show()

In [ ]:
time_273V=np.arange(0,160,1)
plt.figure(figsize=(6,4))
plt.scatter(time_273V,gain_single_pe_273,label='Gain')
# plt.legend(loc=1)
plt.ylim(12,13)
plt.grid()
plt.savefig('gain_fluctuation.pdf')
plt.show()

In [ ]:
pe_corrected_gain=[]
pe_corrected_gain_err=[]
for pe,count in zip(calib_pe_273,calib_count_273):
    plt.subplot(2,2,1)
    plt.plot(pe,count)
    plt.xlim(0,10)
    cut=np.where((pe>0.75) & (pe<1.25))
    popt,pcov=curve_fit(gauss,pe[cut],count[cut],p0=[np.max(count[cut]),1,0.5])
    plt.plot(pe[cut],gauss(pe[cut],*popt),color='r',lw=2,label='Fit')
    # print(popt)
    params_pe = [] 

    for i in range(1,5):
        cut_pe=np.where((pe>i*popt[1]-0.25) & (pe<i*popt[1]+0.25))
        # print(pe[cut_pe])
        popt_pe,pcov_pe=curve_fit(gauss,pe[cut_pe],count[cut_pe],p0=[np.max(count[cut_pe]),i*popt[1],popt[2]])
        plt.plot(pe[cut_pe],gauss(pe[cut_pe],*popt_pe),color='r',lw=2)
        params_pe.append(popt_pe)
        plt.xlim(0,10)
    plt.subplot(2,2,2)
    xdata=np.arange(1,len(params_pe)+1,1)
    plt.scatter(xdata, np.array(params_pe)[:,1], s=10, zorder=10)
    popt_line,pcov_line = curve_fit(line, xdata, np.array(params_pe)[:,1], p0=[10,0] )
    x_plot=np.arange(0,xdata[-1]+1,1)
    plt.plot(x_plot, line(x_plot, *popt_line), color=colors[1], label='Gain: {:.2f} ADC/p.e.'.format(popt_line[0]))
    pe_corrected_gain.append(popt_line[0])
    pe_corrected_gain_err.append(np.sqrt(np.diag(pcov_line)[0]))
    plt.legend(loc=2)
    # plt.xlim(0,10)
    plt.grid()
    plt.xlabel('PE')
    plt.ylabel('Counts')
    # plt.savefig('calibration.pdf')
    plt.show()

In [ ]:
plt.figure(figsize=(6,4))
plt.errorbar(time_273V,pe_corrected_gain,yerr=pe_corrected_gain_err,fmt='o')
plt.ylabel('Normalized PE/PE')
plt.xlabel('Time [minutes]')
plt.ylim(0.8,1.2)
plt.grid()
plt.savefig('corrected_gain_273.pdf')
plt.show()

In [ ]:

color_array,sm=get_color_map(len(calib_pe_273))
n_bins=30 # number of bins to rebin by
n_files=26 #number of files to add together
avg=0

plt.figure(figsize=(8,6))




for i in range(len(calib_pe_273)):

    x,y=rebin_(calib_pe_273[i],calib_count_273[i],n_bins)
    avg+=y
    if (i%n_files==n_files-1):

        # plt.subplot(2,1,1)
        plt.plot(x,avg,lw=3,color=color_array[i])
        plt.title('E= 273 V/cm')
        # plt.yscale('log')
        plt.xlim(20,100)
        plt.ylim(0,600)
        plt.ylabel('Counts')


        # # plt.subplot(2,1,2)
        # plt.plot(x,avg,lw=3,color=color_array[i])
        # # plt.yscale('log')
        # plt.xlim(0,10)
        # # plt.ylim(0,100)
        # plt.ylabel('Counts')
        avg=0
plt.colorbar(sm)
plt.xlabel('PE')
plt.grid()
plt.savefig('peak_shift273.pdf')
plt.show()


In [ ]:

plt.figure(figsize=(6,4))
# plt.errorbar(time_0V,np.array(mu_gauss_list_0V),yerr=np.array(mu_gauss_err_0V),fmt='o',label='0 V/cm')

plt.errorbar(time_273V,np.array(mu_gauss_list_273V),yerr=np.array(mu_gauss_err_273V),fmt='o',label='273 V/cm')

plt.grid()
plt.xlabel('Time[minutes]')
plt.ylabel('$\mu$ [PE]')
# plt.scatter(time_273V,mu_cdf_273,label='CDF',color=colors[2])
plt.legend(loc=1)
plt.ylim(40,47)
plt.savefig('mu_273.pdf')
# plt.title('')
plt.show()


In [ ]:
mu_gauss_total=[]
mu_gauss_err_total=[]

for vals in mu_gauss_list_0V:
    mu_gauss_total.append(vals)
for val in mu_gauss_list_273V:
    mu_gauss_total.append(val)

for va in mu_gauss_err_0V:
    mu_gauss_err_total.append(va)
for v in mu_gauss_err_273V:
    mu_gauss_err_total.append(v)

time_total=np.arange(0,len(mu_gauss_total),1)
plt.figure(figsize=(15,3))
plt.errorbar(time_total[0:59],mu_gauss_total[0:59],yerr=mu_gauss_err_total[0:59],fmt='o',label='0 [V/cm]')
plt.errorbar(time_total[60:],mu_gauss_total[60:],yerr=mu_gauss_err_total[60:],fmt='o',label='273 [V/cm]')
plt.legend(loc=1)
plt.xlabel('Time [minutes]')
plt.ylabel('$\mu$ [PE]')
plt.grid()
plt.savefig('mu_total.pdf')
plt.show()




In [ ]:

n_bins=1 # number of bins to rebin by
avg_0=0
avg_273=0

recal=54.54821181318941/43.25747021259289


plt.figure(figsize=(15,60))
f,ax= plt.subplots(2,1,gridspec_kw={'height_ratios': [10,5]})


for i in range(len(calib_pe_0)):
    x_0,y=rebin_(calib_pe_0[i],calib_count_0[i],n_bins)
    avg_0+=y
# print(len(avg_0))

for i in range(len(calib_pe_273)):
    x_273,y=rebin_(calib_pe_273[i],calib_count_273[i],n_bins)
    avg_273+=y
# print(len(avg_273))

rate_0=avg_0/60
rate_273=avg_273/161
# ax[0].plot(x_273,rate_0,lw=3,label='E = 0 V/cm')
cut_0=np.where((x_273>30) & (x_273<90))
popt_0,pcov_0=curve_fit(gauss,x_273[cut_0],rate_0[cut_0],p0=[np.max(rate_0[cut_0]),65,80])
# ax[0].plot(x_273[cut_0],gauss(x_273[cut_0],*popt_0),color='r',lw=2)
integral_0=np.trapz(gauss(x_273[cut_0],*popt_0),x_273[cut_0])
print('Integral (0 V/cm): ',integral_0)
print('Integral from Data:',np.sum(rate_0[cut_0]))
print(popt_0[1])




# ax[0].plot(x_273,avg_273/(161),lw=3,label='E $\sim$260 V/cm')
cut_273=np.where((x_273>20) & (x_273<80))
popt_273,pcov_273=curve_fit(gauss,x_273[cut_273],rate_273[cut_273],p0=[np.max(rate_273[cut_0]),50,80])
# ax[0].plot(x_273[cut_273],gauss(x_273[cut_273],*popt_273),color='k',lw=2)
integral_260=np.trapz(gauss(x_273[cut_273],*popt_0),x_273[cut_273])
print('Integral (260 V/cm): ',integral_260)
print('Integral from Data:',np.sum(rate_273[cut_273]))
print(popt_273[1])
# ax[0].set_yscale('log')
# ax[0].set_xlim(0,110)
# ax[0].set_ylim(0.1,0.75)
# ax[0].set_ylabel('Rate [Hz]')
# ax[0].set_xlabel('Arb Units')
# ax[0].legend(loc=1)

ax[0].plot(x_273,rate_0,lw=3,label='E = 0 V/cm')
cut=np.where((x_273>0.5) & (x_273<1))
popt,pcov=curve_fit(gauss,x_273[cut],rate_0[cut],p0=[np.max(rate_0[cut]),0.75,0.5])
params_pe_273=[]
params_pe_0=[]
for i in range(1,8):
    cut_pe_0=np.where((x_273>i*popt[1]-0.25) & (x_273<i*popt[1]+0.25))
    popt_pe_no_field,pcov_pe_no_field=curve_fit(gauss,x_273[cut_pe_0],rate_0[cut_pe_0],p0=[np.max(rate_0[cut_pe_0]),i*popt[1],popt[2]])
    # ax[0].plot(x_273[cut_pe_0],gauss(x_273[cut_pe_0],*popt_pe_no_field),color='r',lw=2)
    params_pe_0.append(popt_pe_no_field)


xdata_0 = np.arange(1,len(params_pe_0)+1,1)
plt.scatter(xdata_0, np.array(params_pe_0)[:,1], s=10, zorder=10)
popt_line_0,pcov_line_0 = curve_fit(line, xdata_0, np.array(params_pe_0)[:,1], p0=[10,0] )
x_plot_0=np.arange(0,xdata_0[-1]+1,1)
ax[1].plot(x_plot_0, line(x_plot_0, *popt_line_0), color=colors[0], label='E =0 V/cm, Slope: {:.2f}'.format(popt_line_0[0]))
ax[1].legend(loc=2)

ax[0].plot(x_273,rate_273,lw=3,label='E $\sim$260 V/cm')
cut=np.where((x_273>0.75) & (x_273<1.25))
popt,pcov=curve_fit(gauss,x_273[cut],rate_273[cut],p0=[np.max(rate_273[cut]),1,0.5])
# ax[1].plot(x_273[cut],gauss(x_273[cut],*popt),color='r',lw=2)
ax[0].set_yscale('log')
ax[0].set_xlim(0,10)
ax[0].set_ylim(0.1,100)
ax[0].set_ylabel('Rate [Hz]/'+str(np.round(x_273[1]-x_273[0],2))+'PE')
ax[0].set_xlabel('Arb Units')
ax[0].xaxis.set_ticks(np.arange(0,10,1))
ax[0].legend(loc=1)
ax[0].grid()
for i in range(1,8):
    cut_pe_273=np.where((x_273>i*popt[1]-0.25) & (x_273<i*popt[1]+0.25))
    popt_pe_field,pcov_pe_field=curve_fit(gauss,x_273[cut_pe_273],rate_273[cut_pe_273],p0=[np.max(rate_273[cut_pe_273]),i*popt[1],popt[2]])
    # ax[1].plot(x_273[cut_pe_273],gauss(x_273[cut_pe_273],*popt_pe_field),color='r',lw=2)
    params_pe_273.append(popt_pe_field)


xdata_273 = np.arange(1,len(params_pe_273)+1,1)
plt.scatter(xdata_273, np.array(params_pe_273)[:,1], s=10, zorder=10)
popt_line_273,pcov_line_273 = curve_fit(line, xdata_273, np.array(params_pe_273)[:,1], p0=[10,0] )
x_plot_273=np.arange(0,xdata_273[-1]+1,1)
ax[1].plot(x_plot_273, line(x_plot_273, *popt_line_273), color=colors[1], label='E $\sim$260 V/cm, Slope: {:.2f} '.format(popt_line_273[0]))
ax[1].legend(loc=4)
ax[1].set_xlabel('PE')
ax[1].set_ylabel('Normalized PE/PE')


plt.tight_layout()
plt.grid()
plt.savefig('calibrated_pe_comparison.pdf')
plt.show()
# print(popt_0[1])
# print(popt_273[1])
# print(popt_273[1]/popt_0[1])
print(0.13*100/0.87)

N_photons_0=24835.188420362
N_photons_260=12909.967577528

print(N_photons_260/N_photons_0)


print(x_273[1]-x_273[0])

In [ ]:
n_bins=1 # number of bins to rebin by
avg_0=0
avg_273=0

recal=54.54821181318941/43.25747021259289


plt.figure(figsize=(10,6))
# f,ax= plt.subplots(1,1,gridspec_kw={'height_ratios': [5]})


for i in range(len(calib_pe_0)):
    x_0,y=rebin_(calib_pe_0[i],calib_count_0[i],n_bins)
    avg_0+=y
# print(len(avg_0))

for i in range(len(calib_pe_273)):
    x_273,y=rebin_(calib_pe_273[i],calib_count_273[i],n_bins)
    avg_273+=y
# print(len(avg_273))

rate_0=avg_0/60
rate_273=avg_273/161
plt.plot(x_273,rate_0,lw=3,label='E: 0 V/cm, $\mu$:'+ str(np.round(popt_0[1],2))+'$\pm$'+str(np.round(np.sqrt(np.diag(pcov_0))[1],2)))
cut_0=np.where((x_273>30) & (x_273<90))
popt_0,pcov_0=curve_fit(gauss,x_273[cut_0],rate_0[cut_0],p0=[np.max(rate_0[cut_0]),65,80])
# plt.plot(x_273[cut_0],gauss(x_273[cut_0],*popt_0),color='r',lw=2)
integral_0=np.trapz(gauss(x_273[cut_0],*popt_0),x_273[cut_0])
print('Integral (0 V/cm): ',integral_0)
print('Integral from Data:',np.sum(rate_0[cut_0]))
print(popt_0[1])




plt.plot(x_273,avg_273/(161),lw=3,label='E $\sim$260 V/cm, $\mu$:'+ str(np.round(popt_273[1],2))+'$\pm$'+str(np.round(np.sqrt(np.diag(pcov_273))[1],2)))
cut_273=np.where((x_273>20) & (x_273<80))
popt_273,pcov_273=curve_fit(gauss,x_273[cut_273],rate_273[cut_273],p0=[np.max(rate_273[cut_0]),50,80])
# plt.plot(x_273[cut_273],gauss(x_273[cut_273],*popt_273),color='k',lw=2)
integral_260=np.trapz(gauss(x_273[cut_273],*popt_0),x_273[cut_273])
print('Integral (260 V/cm): ',integral_260)
print('Integral from Data:',np.sum(rate_273[cut_273]))
print(popt_273[1])
# ax[0].set_yscale('log')
plt.xlim(0,110)
plt.ylim(0.1,0.75)
plt.ylabel('Rate [Hz]/'+str(np.round(x_273[1]-x_273[0],2))+'PE')
plt.xlabel('Arb Units')
plt.legend(loc=1)
plt.grid()
plt.savefig('calibrated_133.pdf')
plt.show()